OpenAI ChatGPT APIを使って安価にPDF論文を和訳・要約

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.3/220.3 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [ ]:
!pip install pdfminer.six

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 41.5 MB/s eta 0:00:00


In [ ]:
import argparse
import re
import numpy as np
import yaml

import openai
from pdfminer.high_level import extract_text

In [ ]:
# OpenAIのAPIキーを設定(1)
#openai.api_key = ""

In [ ]:
# OpenAIのAPIキーを設定(2)

f = open('gptkey.txt','r')
ai_key = f.read()
f.close()

client = openai.OpenAI(
    api_key = ai_key
)


In [ ]:

# APIへ送信するワード数の上限を、経験的に設定する
WC_MAX = int(4096 * 7 / 16)

# PDFファイルからテキスト情報を抽出する
def extract_text_from_pdf(file_path) :
    text = extract_text(file_path)
    return text

# テキスト情報から単語途中で現れるハイフンと改行を削除する
def clean_extracted_text(text) :
    text = text.replace('-\n', '')
    text = re.sub(r'\s+', ' ', text)
    return text


# APIへ送信するメッセージ（プロンプト）を作成する
def create_prompt(text, total_num_req) :
    return f"""英語の研究論文の一部を日本語で要約するタスクを行います。
研究論文は全部で{total_num_req + 1}個に分割しています。
以下のルールに従ってください。

・リスト形式で出力する (先頭は - を使う)
・簡潔に表現する
・不明な単語や人名と思われるものは英語のまま表示する

それでは開始します。

英語の論文の一部:
{text}

日本語で要約した文章:"""

# テキストをWC_MAXごとに分割する
def split_text(text, wc_max) :
    '''
    :param text: english oneline sentence
    :param wc_max:  word count max (if token max is 4097, it should be 2048)
    :return: chunk[]
    '''
    words = text.split()  # Split the text into words
    chunks = [' '.join(words[i :i + wc_max]) for i in range(0, len(words), wc_max)]
    return chunks

# main()メソッド
def main() :
    # PDFファイルからテキスト情報を抽出して整形する
    extracted_text = extract_text_from_pdf('/content/2939672.2939778.pdf')
    clean_text = clean_extracted_text(extracted_text)

    # テキスト情報の概要を表示する。APIコール回数を表示する。
    print(f"input file has {len(clean_text.split())} words, "
          f"so API requests repeat {np.ceil(len(clean_text.split()) / WC_MAX)} times.")

    # 分割処理（4,097トークン以下に抑えるための処理）
    splited_clean_text = split_text(clean_text, WC_MAX)

    # 分割したテキストごとに要約をリクエストする
    for i in range(len(splited_clean_text)) :
        print(f"{i} times request... ({len(splited_clean_text[i].split())} words are included)")
        # Setup prompt messages
        messages = [
            {"role" : "system",
             "content" : "あなたは、金融機関に所属するサイバーセキュリティ研究者です。あなたは、前後に問い合わせした内容を考慮して思慮深い回答をします。"},
            {"role" : "user", "content" : create_prompt(splited_clean_text[i], len(splited_clean_text))}
        ]

        # APIにリクエストを送信する
        # token数超過などの例外を返す可能性があるため、try/exceptを設定しておく。
        try:
            response = client.chat.completions.create(
                model="gpt-3.5-turbo",  # GPTのモデルを指定。安価なgpt-3.5-turbo
                messages=messages,
                max_tokens=800,  # 生成するトークンの最大数（入力と合算して4097以内に収める必要あり）
                n=1,  # 生成するレスポンスの数
                stop=None,  # 停止トークンの設定
                temperature=0.7,  # 生成時のランダム性の制御
                top_p=1,  # トークン選択時の確率閾値
            )
            # 生成するレスポンス数は1という前提
            print(response.choices[0].message.content)
        except openai.error.InvalidRequestError as e:
            print("OpenAI API error occurred: " + str(e))
main()



input file has 9396 words, so API requests repeat 6.0 times.
0 times request... (1792 words are included)
- 論文のタイトルは「なぜ私はあなたを信頼すべきか？」で、予測の説明方法について述べています。
- この研究では、予測の理由を説明することが信頼性の評価に非常に重要であり、新しいモデルを展開する際にも重要であることを示しています。
- 著者たちは、LIMEという新しい説明技術を提案しており、予測の周りで局所的に解釈可能なモデルを学習することで、任意の分類器の予測を説明する方法を示しています。
- また、代表的な個別の予測とその説明を非冗長な方法で提示することにより、モデルの説明方法を提案しています。
- さまざまなテキスト（ランダムフォレストなど）や画像分類（ニューラルネットワークなど）のための異なるモデルを説明する柔軟性を示しています。
- 新しい実験を通じて、説明の有用性を示しており、予測を信頼すべきかどうかを判断する、モデルを選択する、信頼性の低い分類器を改善する、なぜ分類器を信頼すべきでないかを特定するなど、さまざまなシナリオでの結果を示しています。
1 times request... (1792 words are included)
- LIMEは、モデルに対してローカルに忠実でありながら解釈可能な説明を提供するための手法である。
- 解釈可能なデータ表現は、実際のモデルが使用する特徴とは異なる人間に理解可能な表現である。
- LIMEは、近傍のインスタンスをサンプリングし、それらに対してモデルの予測を行い、重み付けすることで、ローカルな振る舞いを学習する。
- LIMEは、疎な線形モデルを用いて説明を生成するが、他のモデルや複雑なモデルにも適用可能である。
- LIMEの具体的な例として、テキスト分類と画像分類の説明が示されている。
2 times request... (1792 words are included)
- アルゴリズム1は、個々の予測の説明を生成するため、データセットのサイズには依存せず、f（x）の計算時間とサンプルの数Nに依存します。
- ランダムフォレストの説明（scikit-learnを使用）は、N = 5000のラ